In [21]:
%%capture
!pip install xarray rioxarray geopandas regionmask  dask netCDF4 bottleneck rasterio

In [22]:
import xarray as xr
import rioxarray as rx
import regionmask
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio

In [2]:
netcdfPath  = 'data/Prec_ACCESS-ESM1-5_ssp126_2021_2040.nc'
africaShapeFile = 'data/Africa/Africa_Boundaries.shp'

In [3]:
futureScenariosData = xr.open_dataset(netcdfPath)
futureScenariosData = futureScenariosData.sel(lat=slice(37,-35), lon=slice(-18, 51))

In [12]:
timeArray = futureScenariosData.time.values

In [13]:
futureScenariosData = futureScenariosData.sel(time=timeArray[0])

In [16]:
data = futureScenariosData.tmin
dataValues = data.values

In [26]:
data

<xarray.DataArray 'tmin' (band: 1, lat: 1440, lon: 1380)> Size: 8MB
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 12kB 36.97 36.92 36.87 36.82 ... -34.88 -34.93 -34.98
  * lon      (lon) float64 11kB -17.97 -17.92 -17.87 ... 50.88 50.93 50.98
    time     datetime64[ns] 8B 2021-01-31
Dimensions without coordinates: band
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  129.73481750488
    STATISTICS_MEAN:     -9999
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   -9999
    grid_mapping:        spatial_ref
    long_name:           chirps-v2.0.1995.01.01

In [28]:
with rasterio.open("data/temp.tif", "w", driver="GTiff",
                  width=dataValues.shape[1], height=dataValues.shape[0],
                  count=1, dtype=dataValues.dtype) as dst:
    dst.write(dataValues)  # Write the data to the first band



In [30]:
# Convert the GeoTIFF to ASCII using rasterio
with rasterio.open("data/temp.tif") as src:
    array = src.read(1)  # Read the first band
    with open("data/sample.asc", "w") as f:
        for row in array:
            f.write(" ".join(map(str, row)))  # Write each row
            f.write("\n")  # Add newline

# Remove the temporary GeoTIFF file


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [20]:
help(rx)

Help on package rioxarray:

NAME
    rioxarray - Top-level package for rioxarray.

PACKAGE CONTENTS
    _io
    _options
    _show_versions
    crs
    exceptions
    merge
    raster_array
    raster_dataset
    raster_writer
    rioxarray
    xarray_plugin

CLASSES
    builtins.object
        rioxarray._options.set_options
    
    class set_options(builtins.object)
     |  set_options(**kwargs)
     |  
     |  Set the global rioxarray option.
     |  
     |  .. versionadded:: 0.3.0
     |  .. versionadded:: 0.7.0 skip_missing_spatial_dims
     |  
     |  Parameters
     |  ----------
     |  export_grid_mapping: bool, default=True
     |      If True, this option will export the full Climate and Forecasts (CF)
     |      grid mapping attributes for the CRS. This is useful if you are exporting
     |      your file to netCDF using :meth:`xarray.Dataset.to_netcdf()`. When disabled,
     |      only the ``crs_wkt`` and ``spatial_ref`` attributes will be written and the
     |      